In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/zindi/crop

/content/drive/MyDrive/Colab Notebooks/zindi/crop


### <font color="brown"> Import packages

In [ ]:
!pip -qq install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 2.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import random
import math
from joblib import Parallel, delayed

from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Set seed for reproducability
SEED = 2023
random.seed(SEED)
np.random.seed(SEED)

### <font color="brown"> Import data

In [ ]:
# import data for Iran - 12 months (2019-07-01 to 2020-06-30)

# Sentinel-2 (S2) data
train_data_iran1 = pd.read_csv('train_data_iran_2019_2020_month_0to100.csv')
train_data_iran2 = pd.read_csv('train_data_iran_2019_2020_month_100to200.csv')
train_data_iran3 = pd.read_csv('train_data_iran_2019_2020_month_200to300.csv')
train_data_iran4 = pd.read_csv('train_data_iran_2019_2020_month_300to400.csv')
train_data_iran5 = pd.read_csv('train_data_iran_2019_2020_month_400to500.csv')
train_data_iran_sentinel2 = pd.concat([train_data_iran1,train_data_iran2,train_data_iran3,train_data_iran4,train_data_iran5], axis=0).reset_index()

test_data_iran1 = pd.read_csv('test_data_iran_2019_2020_month_0to100.csv')
test_data_iran2 = pd.read_csv('test_data_iran_2019_2020_month_100to200.csv')
test_data_iran3 = pd.read_csv('test_data_iran_2019_2020_month_200to300.csv')
test_data_iran4 = pd.read_csv('test_data_iran_2019_2020_month_300to400.csv')
test_data_iran5 = pd.read_csv('test_data_iran_2019_2020_month_400to500.csv')
test_data_iran_sentinel2 = pd.concat([test_data_iran1,test_data_iran2,test_data_iran3,test_data_iran4,test_data_iran5], axis=0).reset_index()

# Sentinel-1 (S1) data
train_data_iran_sentinel1 = pd.read_csv('train_data_iran_sentinel1.csv')
test_data_iran_sentinel1 = pd.read_csv('test_data_iran_sentinel1.csv')

train_data_iran_S1S2 = pd.concat([train_data_iran_sentinel2, train_data_iran_sentinel1.iloc[:,4:]], axis=1)
test_data_iran_S1S2 = pd.concat([test_data_iran_sentinel2, test_data_iran_sentinel1.iloc[:,3:]], axis=1)

# Landsat-8 (L8) data
train_data_iran1 = pd.read_csv('train_data_iran_LANDSAT8_0to100.csv')
train_data_iran2 = pd.read_csv('train_data_iran_LANDSAT8_100to500.csv')
train_data_iran_landsat = pd.concat([train_data_iran1, train_data_iran2], axis=0).reset_index()
test_data_iran_landsat = pd.read_csv('test_data_iran_LANDSAT8.csv')

In [ ]:
# import data for Sudan - 12 months (2019-07-01 to 2020-06-30)

# Sentinel-2 data
train_data_sudan1 = pd.read_csv('train_data_sudan_2019_2020_month_0to100.csv')
train_data_sudan2 = pd.read_csv('train_data_sudan_2019_2020_month_100to200.csv')
train_data_sudan3 = pd.read_csv('train_data_sudan_2019_2020_month_200to300.csv')
train_data_sudan4 = pd.read_csv('train_data_sudan_2019_2020_month_300to400.csv')
train_data_sudan5 = pd.read_csv('train_data_sudan_2019_2020_month_400to500.csv')
train_data_sudan_sentinel2 = pd.concat([train_data_sudan1,train_data_sudan2,train_data_sudan3,train_data_sudan4,train_data_sudan5], axis=0).reset_index()

test_data_sudan1 = pd.read_csv('test_data_sudan_2019_2020_month_0to100.csv')
test_data_sudan2 = pd.read_csv('test_data_sudan_2019_2020_month_100to200.csv')
test_data_sudan3 = pd.read_csv('test_data_sudan_2019_2020_month_200to300.csv')
test_data_sudan4 = pd.read_csv('test_data_sudan_2019_2020_month_300to400.csv')
test_data_sudan5 = pd.read_csv('test_data_sudan_2019_2020_month_400to500.csv')
test_data_sudan_sentinel2 = pd.concat([test_data_sudan1,test_data_sudan2,test_data_sudan3,test_data_sudan4,test_data_sudan5], axis=0).reset_index()

# Sentinel-1 data
train_data_sudan_sentinel1 = pd.read_csv('train_data_sudan_sentinel1.csv')
test_data_sudan_sentinel1 = pd.read_csv('test_data_sudan_sentinel1.csv')

train_data_sudan_S1S2 = pd.concat([train_data_sudan_sentinel2, train_data_sudan_sentinel1.iloc[:,4:]], axis=1)
test_data_sudan_S1S2 = pd.concat([test_data_sudan_sentinel2, test_data_sudan_sentinel1.iloc[:,3:]], axis=1)

# Landsat-8 data
train_data_sudan1 = pd.read_csv('train_data_sudan_LANDSAT8_0to250.csv')
train_data_sudan2 = pd.read_csv('train_data_sudan_LANDSAT8_250to500.csv')
train_data_sudan_landsat = pd.concat([train_data_sudan1, train_data_sudan2], axis=0).reset_index()

test_data_sudan1 = pd.read_csv('test_data_sudan_LANDSAT8_0to250.csv')
test_data_sudan2 = pd.read_csv('test_data_sudan_LANDSAT8_250to500.csv')
test_data_sudan_landsat = pd.concat([test_data_sudan1, test_data_sudan2], axis=0).reset_index()

In [ ]:
# import data for Afghanistan

# Sentinel-2 data

## 1 month (2022-04-01 to 2022-04-30)
train_data_afghanistan_sentinel2_april = pd.read_csv('train_data_afghanistan_april.csv')
test_data_afghanistan_sentinel2_april = pd.read_csv('test_data_afghanistan_april.csv')

## 12 months (2021-04-01 to 2022-03-31)
train_data_afghanistan1 = pd.read_csv('train_data_afghanistan_2021_month_0to100.csv')
train_data_afghanistan2 = pd.read_csv('train_data_afghanistan_2021_month_100to200.csv')
train_data_afghanistan3 = pd.read_csv('train_data_afghanistan_2021_month_200to300.csv')
train_data_afghanistan4 = pd.read_csv('train_data_afghanistan_2021_month_300to400.csv')
train_data_afghanistan5 = pd.read_csv('train_data_afghanistan_2021_month_400to500.csv')
train_data_afghanistan_sentinel2 = pd.concat([train_data_afghanistan1,train_data_afghanistan2,train_data_afghanistan3,train_data_afghanistan4,train_data_afghanistan5], axis=0).reset_index()

test_data_afghanistan1 = pd.read_csv('test_data_afghanistan_2021_month_0to100.csv')
test_data_afghanistan2 = pd.read_csv('test_data_afghanistan_2021_month_100to200.csv')
test_data_afghanistan3 = pd.read_csv('test_data_afghanistan_2021_month_200to300.csv')
test_data_afghanistan4 = pd.read_csv('test_data_afghanistan_2021_month_300to400.csv')
test_data_afghanistan5 = pd.read_csv('test_data_afghanistan_2021_month_400to500.csv')
test_data_afghanistan_sentinel2 = pd.concat([test_data_afghanistan1,test_data_afghanistan2,test_data_afghanistan3,test_data_afghanistan4,test_data_afghanistan5], axis=0).reset_index()

# Sentinel-1 data

## 1 month (2022-04-01 to 2022-04-30)
train_data_afghanistan_sentinel1_april = pd.read_csv('train_data_afghanistan_sentinel1.csv')
test_data_afghanistan_sentinel1_april = pd.read_csv('test_data_afghanistan_sentinel1.csv')

## 12 months (2021-04-01 to 2022-03-31)
train_data_afghanistan_sentinel1 = pd.read_csv('train_data_afghanistan_2021_month_sentinel1.csv')
test_data_afghanistan_sentinel1 = pd.read_csv('test_data_afghanistan_2021_month_sentinel1.csv')

# Sentinel-1 and Sentinel-2 data for 1 month (2022-04-01 to 2022-04-30)
train_data_afghanistan_S1S2_april = pd.concat([train_data_afghanistan_sentinel2_april, train_data_afghanistan_sentinel1_april.iloc[:,4:]], axis=1)
test_data_afghanistan_S1S2_april = pd.concat([test_data_afghanistan_sentinel2_april, test_data_afghanistan_sentinel1_april.iloc[:,3:]], axis=1)

# Sentinel-1 and Sentinel-2 data for 12 months (2021-04-01 to 2022-03-31)
train_data_afghanistan_S1S2 = pd.concat([train_data_afghanistan_sentinel2, train_data_afghanistan_sentinel1.iloc[:,4:]], axis=1)
test_data_afghanistan_S1S2 = pd.concat([test_data_afghanistan_sentinel2, test_data_afghanistan_sentinel1.iloc[:,3:]], axis=1)

# Landsat-8 data for 12 months (2021-04-01 to 2022-03-31)
train_data_afghanistan1 = pd.read_csv('train_data_afghanistan_LANDSAT8_0to250.csv')
train_data_afghanistan2 = pd.read_csv('train_data_afghanistan_LANDSAT8_250to500.csv')
train_data_afghanistan_landsat = pd.concat([train_data_afghanistan1, train_data_afghanistan2], axis=0).reset_index()

test_data_afghanistan_landsat = pd.read_csv('test_data_afghanistan_LANDSAT8.csv')

### <font color="brown"> Functions Utilized

In [ ]:
# Create Features (or remote sensing indices) for 12 months

eps=1e-08

def vegetation_index(i):

  b01 = comb.filter(like = "B1_").values[:,i]
  b02 = comb.filter(like = "B2_").values[:,i]
  b03 = comb.filter(like = "B3_").values[:,i]
  b04 = comb.filter(like = "B4_").values[:,i]
  b05 = comb.filter(like = "B5_").values[:,i]
  b06 = comb.filter(like = "B6_").values[:,i]
  b07 = comb.filter(like = "B7_").values[:,i]
  b08 = comb.filter(like = "B8_").values[:,i]
  b8a = comb.filter(like = "B8A_").values[:,i]
  b09 = comb.filter(like = "B9_").values[:,i]
  b11 = comb.filter(like = "B11_").values[:,i]
  b12 = comb.filter(like = "B12_").values[:,i]

  MI = (b8a - b11) / (b8a + b11+ eps)
  ARVI_ =  (b08 - (2 * b04) + b02) / (b08 + (2 * b04) - b02+ eps)
  SIPI = (b08 - b02) / (b08 - b04+ eps)
  RENDVI = (b06 - b05) / (b06 + b05+ eps)
  MRESR = (b06 - b01) / (b05 - b01+ eps)
  RYI = b03 / (b02+ eps)
  NDYI = (b03 - b02) / (b03 + b02+ eps)
  DYI = b03 - b02
  ACI = b08 * (b04 + b03)
  AVI_ = (b08 * (1 - b04) * (b08 - b04))
  SI = ((1 - b02) * (1 - b03) * (1 - b04))
  BSI = ((b11 + b04) - (b08 + b02)) / ((b11 + b04) + (b08 + b02)+ eps)
  L = 0.33; SAVI = ((b08 - b04) / (b08 + b04 + L)) * (1 + L)
  FIDET = b12 / (b8a * b09+ eps)
  MTCI_ = (b06 - b05) / (b05 - b04+ eps)
  NPCRI = (b04 - b02) / (b04 + b02+ eps)
  S2REP = 705 + 35 * ((((b07 + b04) / 2) - b05) / (b06 - b05+ eps))
  CCCI = ((b08 - b05) / (b08 + b05+ eps)) / ((b08 - b04+ eps) / (b08 + b04+ eps))
  MCARI = ((b05 - b04) - 0.2 * (b05 - b03)) * (b05 / (b04+ eps))
  TCARI = 3 * ((b05 - b04) - 0.2 * (b05 - b03) * (b05 / (b04+ eps)))
  PVI = (b08 - 0.3 * b04 - 0.5) / ((1 + 0.3 * 2) ** (1 / 2.0))
  EVI = 2.5 * (b08 - b04) / (b08 + 6 * b04 - 7.5 * b02 + 1)
  EVI2 = 2.4 * (b08 - b04) / (b08 + b04 + 1)
  NDVI = ((b08 - b04) / (b08 + b04+ eps))
  NDVI2 = ((b12 - b08) / (b12 + b08+ eps))
  MNDVI = (b08 - b04) / (b08 + b04 - 2 * b02+ eps)
  BAI = 1 / ((0.1 - b04) ** 2 + (0.06 - b08) ** 2)
  NDSI = (b03 - b11) / (b03 + b11+ eps)
  MRENDVI = (b06 - b05) / (b06 + b05 - 2 * b01+ eps)
  NDVIre = (b08 - b05) / (b08 + b05+ eps)
  CIre = ((b08 / (b05+ eps)) - 1)
  NDMI = (b08 - b11) / (b08 + b11+ eps)
  TNDVI = [(x) ** (1 / 2.0) for x in ((b08 - b04) / (b08 + b04) + 0.5)]
  SWIRmin = 0.378; SWIRmax = 0.397; SWIIRmin = 0.027; NDVIc = (b08 - b04) / (b08 + b04) * (1.0 - (b12 - SWIIRmin) / (SWIRmax - SWIRmin))
  VDVI = (2 * b03 - b04 - b02) / (2 * b03 + b04 + b02+ eps)
  TVI = (120 * (b06 - b03) - 200 * (b04 - b03)) / 2
  EXG = 2 * b03 - b04 - b02
  PSRI = (b04 - b02) / (b06+ eps)
  RDVI = [(i - j) / (i + j) ** (1 / 2.0) for i, j in zip(b08, b04)]
  RATIO1 = b01 / (b03+ eps)
  RATIO2 = b01 / (b05+ eps)
  RATIO3 = b11 / (b12+ eps)
  RATIO4 = b05 / (b04+ eps)
  NDWI = (b03 - b08) / (b03 + b08+ eps)
  NDRE = (b09 - b05) / (b09 + b05+ eps)
  NDRE_ = (b05 - b04) / (b05 + b04+ eps)
  NDRE7 = (b07 - b04) / (b07 + b04+ eps)
  MSI = b11 / (b8a+ eps)
  RECI = (b08 / (b04+ eps)) - 1
  VARI = (b03 - b04) / (b03 + b04 - b02+ eps)
  MTCI = (b8a - b06) / (b07 + b06+ eps)
  ExBlue = (2 * b02) - (b03 + b04)
  WDRVI = (0.1 * b08 - b04) / (0.1 * b08 + b04+ eps)
  GRNDVI = (b08 - (b03 + b04)) / (b08 + (b03 + b04)+ eps)
  LCI = (b08 - b05) / (b08 + b04+ eps)
  BNDVI = (b08 - b02) / (b08 + b02+ eps)
  AFRI = b08 - 0.66 * (b11 / (b08 + 0.66 * b11+ eps))
  aci = b08 * (b04 + b03)
  bai = 1 / ((0.1 - b04) ** 2 + (0.06 - b08) ** 2)
  NLI = (b08 ** 2 - b04) / (b08 ** 2 + b04+ eps)
  b7b5 = (b07 - b05) / (b07 + b05+ eps)
  b7b6 = (b07 - b06) / (b07 + b06+ eps)
  b8ab5 = (b8a - b05) / (b8a + b05+ eps)
  ATSAVI = 1.22 * (b08 - 1.22 * b04 - 0.03) / (1.22 * b08 + b04 - 1.22 * 0.03 + 0.08 * (1.0 + np.power(1.22, 2.0)))
  AFRI2100 = b08 - 0.5 * b12 / (b08 + 0.56 * b12+ eps)
  ARI = 1.0 / (b03+ eps) - 1.0 / (b05+ eps)
  AVI = 2.0 * b09 - b04
  yy = 0.106; ARVI = (b8a - b04 - yy * (b04 - b02)) / (b8a + b04 - yy * (b04 - b02)+ eps)
  ARVI2 = -0.18 + 1.17 * ((b08 - b04) / (b08 + b04+ eps))
  BWDRVI = (0.1 * b08 - b02) / (0.1 * b08 + b02+ eps)
  BRI = (1.0 / (b03+ eps) - 1.0 / (b05+ eps)) / (b08+ eps)
  CARI = (b05 / (b04+ eps)) * (np.sqrt(np.power(((b05 - b03) / 150.0 * 670.0 + b04 + (b03 - ((b05 - b03) / 150.0 * 550.0))), 2.0))) / (np.power(((b05 - b03) / np.power(150.0, 2.0) + 1.0), 0.5))
  aaa = 0.496; CARI2 = (np.abs(((b05 - b03) / 150.0 * b04 + b04 + b03 - (aaa * b03))) / np.power((np.power(aaa, 2.0) + 1.0), 0.5)) * (b05 / b04)
  Chlgreen = np.power((b07 / (b03+ eps)), (-1.0))
  CIgreen = b08 / (b03+ eps) - 1.0
  Chlrededge = np.power((b07 / b05), (-1.0))
  CVI = b08 * b04 / np.power(b03, 2.0)
  CI = (b04 - b02) / (b04+ eps)
  CTVI = (((b04 - b03) / (b04 + b03)) + 0.5) / np.abs(((b04 - b03) / (b04 + b03)) + 0.5) * np.sqrt(np.abs((((b04 - b03) / (b04 + b03))) + 0.5))
  CRI550 = np.power(b02, (-1.0)) - np.power(b03, (-1.0))
  CRI700 = np.power(b02, (-1.0)) - np.power(b05, (-1.0))
  Datt1 = (b08 - b05) / (b08 - b04+ eps)
  Datt4 = b04 / ((b03 * b05)+ eps)
  Datt6 = b8a / ((b03 * b05)+ eps)
  D678500 = b04 - b02
  D800550 = b08 - b03
  D800680 = b08 - b04
  DVIMSS = 2.4 * b09 - b04
  EVI2_ = 2.5 * (b08 - b04) / (b08 + 2.4 * b04 + 1.0)
  aa = 0.331; bb = 0.329; EPI = aa * b04 / np.power((b03 * b05), bb)
  Fe2 = b12 / (b08+ eps) + b03 / (b04+ eps)
  Fe3 = b04 / (b03+ eps)
  Fo = b11 / (b08+ eps)
  GEMI = ((2.0 * (np.power(b08, 2.0) - np.power(b04, 2.0)) + 1.5 * b08 + 0.5 * b04) / (b08 + b04 + 0.5) * (1.0 - 0.25 * (2.0 * (np.power(b08, 2.0) - np.power(b04, 2.0)) + 1.5 * b08 + 0.5 * b04) / (b08 + b04 + 0.5)) - (b04 - 0.125) / (1.0 - b04))
  GVMI = ((b08 + 0.1) - (b12 + 0.02)) / ((b08 + 0.1) + (b12 + 0.02+ eps))
  Gossan = b11 / (b04+ eps)
  GARI = (b08 - (b03 - (b02 - b04))) / (b08 - (b03 + (b02 - b04))+ eps)
  GLI = (2.0 * b03 - b04 - b02) / (2.0 * b03 + b04 + b02+ eps)
  YY = 0.120; GOSAVI = (b08 - b03) / (b08 + b03 + YY)
  L = 0.482; GSAVI = (b08 - b03) / (b08 + b03 + L) * (1.0 + L)
  GBNDVI = (b08 - (b03 + b02)) / (b08 + (b03 + b02)+ eps)
  H = np.arctan((2.0 * b04 - b03 - b02) / 30.5 * (b03 - b02+ eps))
  a = 0.393; b = 0.809; IVI = (b08 - b) / (a * b04+ eps)
  IPVI = (b09 / (b09 + b05+ eps)) / 2 * (((b05 - b03) / (b05 + b03+ eps)) + 1)
  I = (1.0 / (30.5)) * (b04 + b03 + b02)
  IR550 = np.power(b03, (-1.0))
  IR700 = np.power(b05, (-1.0))
  MIDIR = 0.101; LWCI = np.log(1.0 - (b08 - MIDIR)) / (-np.log(1.0 - (b08 - MIDIR)))
  LogR = np.log(b08 / (b04+ eps))
  Maccioni = (b07 - b05) / (b07 - b04+ eps)
  mCRIG = (np.power(b02, (-1.0)) - np.power(b03, (-1.0))) * b08
  mCRIRE = (np.power(b02, (-1.0)) - np.power(b05, (-1.0))) * b08
  MVI = b09 / (b11+ eps)
  MGVI = -0.386 * b03 - 0.53 * b04 + 0.535 * b06 + 0.532 * b09
  MNSI = 0.404 * b03 - 0.039 * b04 - 0.505 * b06 + 0.762 * b09
  MSBI = 0.406 * b03 + 0.6 * b04 + 0.645 * b06 + 0.243 * b09
  MYVI = 0.723 * b03 - 0.597 * b04 + 0.206 * b06 - 0.278 * b09
  mND680 = (b08 - b04) / (b08 + b04 - 2.0 * b01+ eps)
  mARI = (np.power(b03, (-1.0)) - np.power(b05, (-1.0))) * b08
  MCARI1 = 1.2 * (2.5 * (b08 - b04) - 1.3 * (b08 - b03))
  MCARI2 = 1.5 * (2.5 * (b08 - b04) - 1.3 * (b08 - b03)) / np.sqrt(np.power((2.0 * b08 + 1.0), 2.0) - (6.0 * b08 - 5.0 * np.sqrt(b04)) - 0.5)
  MCARIMTVI2 = (((b05 - b04) - 0.2 * (b05 - b03)) * (b05 / (b04+ eps))) / (1.5 * (1.2 * (b08 - b03) - 2.5 * (b04 - b03)) / np.sqrt(np.power((2.0 * b08 + 1.0), 2.0) - (6.0 * b08 - 5.0 * np.sqrt(b04)) - 0.5))
  MCARIOSAVI = ((b05 - b04) - 0.2 * (b05 - b03) * (b05 / (b04+ eps))) / ((1.0 + 0.16) * (b08 - b04) / (b08 + b04 + 0.16))
  mSR = (b08 - b01) / (b04 - b01+ eps)
  MSRNirRed = ((b08 / (b04+ eps)) - 1.0) / np.sqrt((b08 / (b04+ eps)) + 1.0)
  MSAVI = (2.0 * b08 + 1.0 - np.sqrt(np.power((2.0 * b08 + 1.0), 2.0) - 8.0 * (b08 - b04))) / 2.0
  MSAVIhyper = (0.5) * ((2.0 * b08 + 1.0) - np.sqrt(np.power((2.0 * b08 + 1.0), 2.0) - 8.0 * (b08 - b04)))
  MTVI1 = 1.2 * (1.2 * (b08 - b03) - 2.5 * (b04 - b03))
  MTVI2 = 1.5 * (1.2 * (b08 - b03) - 2.5 * (b04 - b03)) / np.sqrt(np.power((2.0 * b08 + 1.0), 2.0) - (6.0 * b08 - 5.0 * np.sqrt(b04)) - 0.5)
  NormG = b03 / (b08 + b04 + b03+ eps)
  NormR = b04 / (b08 + b04 + b03+ eps)
  NormNIR = b08 / (b08 + b04 + b03+ eps)
  PPR = (b03 - b01) / (b03 + b01+ eps)
  PVR = (b03 - b04) / (b03 + b04+ eps)
  GNDVIhyper = (b07 - b03) / (b07 + b03+ eps)
  OSAVI = (1.0 + 0.16) * (b08 - b04) / (b08 + b04 + 0.16)
  PNDVI = (b08 - (b03 + b04 + b02)) / (b08 + (b03 + b04 + b02)+ eps)
  a = 0.149; ar = 0.374; b = 0.735; PVI_ = (1.0 / np.sqrt(a ** 2.0 + 1.0)) * (b08 - ar - b)
  r700 = 0.715; r675 = 0.722; RARSa1 = (b04 / b05) / (r675 / r700)
  r700 = 0.989; r680 = 0.848; RARSa2 = (b04 / b05) / (r680 / r700)
  r670 = 0.382; r800 = 0.935; RARSa3 = (b04 / b08) / (r670 / r800)
  r680 = 0.523; r800 = 0.866; RARSa4 = (b04 / b08) / (r680 / r800)
  r500 = 0.535; r800 = 0.712; RARSc3 = (b08 / b02) / (r800 / r500)
  r470 = 0.503; r800 = 0.153; RARSc4 = (b08 / b02) / (r800 / r470)
  RBNDVI = (b08 - (b04 + b02)) / (b08 + (b04 + b02)+ eps)
  REIP1 = 700.0 + 40.0 * ((((b04 + b07) / 2.0) - b05) / (b06 - b05+ eps))
  REIP2 = 702.0 + 40.0 * ((((b04 + b07) / 2.0) - b05) / (b06 - b05+ eps))
  Rre = (b04 + b07) / 2.0
  MIRmin = 0.259; MIRmax = 0.640; RSR = b08 / b04 * MIRmax - b12 / MIRmax - MIRmin
  L = 0.781; SAVImir = (b08 - b12) * (1.0 + L) / (b08 + b12 + L)
  IF = (2.0 * b04 - b03 - b02) / (b03 - b02+ eps)
  SR440740 = b01 / (b06+ eps)
  SR520670 = b02 / (b04+ eps)
  SR700 = 1.0 / (b05+ eps)
  SR735710 = b06 / (b05+ eps)
  SR774677 = b07 / (b04+ eps)
  PSSRc1 = b08 / (b02+ eps)
  SR8002170 = b08 / (b12+ eps)
  SR860550 = b8a / (b03+ eps)
  SR860708 = b8a / (b05+ eps)
  SRMIRRed = b12 / (b04+ eps)
  SWIRI = 0.887; SRSWIRINIR = SWIRI / (b08+ eps)
  a = 0.064; b = 0.918; SAVI2 = b08 / (b04 + b / a)
  y = 0.735; Rr = 0.740; L = 0.487; RB = 0.560; SARVI = (1.0 + L) * (b08 - (Rr - y * (RB - Rr))) / (b08 + -(Rr - y * (RB - Rr)) + L)
  SLAVI = b08 / (b04 + b12+ eps)
  SQRT_ = np.sqrt(b08 / b04+ eps)
  SIPI_ = (b08 - b01) / (b08 - b04+ eps)
  GVIMSS = -0.283 * b03 - 0.66 * b04 + 0.577 * b06 + 0.388 * b09
  NSIMSS = -0.016 * b03 + 0.131 * b04 - 0.425 * b06 + 0.882 * b09
  SBIMSS = 0.332 * b03 + 0.603 * b04 + 0.675 * b06 + 0.262 * b09
  GVI = -0.2848 * b02 - 0.2435 * b03 - 0.5436 * b04 + 0.7243 * b08 + 0.084 * b11 - 0.18 * b12
  WET = 0.1509 * b02 + 0.1973 * b03 + 0.3279 * b04 + 0.3406 * b08 - 0.7112 * b11 - 0.4572 * b12
  YVIMSS = -0.899 * b03 + 0.428 * b04 + 0.076 * b06 - 0.041 * b09
  X = 0.114; A = 0.824; B = 0.421; TSAVI = (B * (b08 - B * b04 - A)) / (b04 + B * (b08 - A) + X * (1.0 + np.power(B, 2.0))+ eps)
  a = 0.419; b = 0.787; TSAVI2 = (a * b08 - a * b04 - b) / (b04 + a * b08 - a * b+ eps)
  TVI_ = np.sqrt((((b04 - b03) / (b04 + b03+ eps))) + 0.5)
  TCI = 1.2 * (b05 - b03) - 1.5 * (b04 - b03) * np.sqrt(b05 / (b04+ eps))
  VARI700 = (b05 - 1.7 * b04 + 0.7 * b02) / (b05 + 2.3 * b04 - 1.3 * b02+ eps)
  a = 0.752; WDVI = b08 - a * b04
  EPIChlb = 0.0337 * np.power((b04 / (b03+ eps)), 1.8695)
  LAI_green = 5.405 * b8ab5 - 0.114

  return [MI, ARVI_, SIPI, RENDVI, MRESR, RYI, NDYI, DYI, ACI, AVI_, SI, BSI, SAVI, FIDET, MTCI_, NPCRI, S2REP, CCCI, MCARI, TCARI,\
          PVI, EVI, NDVI, BAI, NDSI, MRENDVI, NDVIre, CIre, NDMI, TNDVI, VDVI, NDVIc, TVI, EXG, PSRI, RDVI, RATIO1, RATIO2, RATIO3, RATIO4,\
          NDVI2, NDWI, NDRE, NDRE_, NDRE7, MSI, RECI, b7b5, b7b6, b8ab5, VARI, MTCI, ExBlue, WDRVI, GRNDVI, LCI, BNDVI, AFRI, aci, bai,\
          NLI, EVI2, ATSAVI, AFRI2100, ARI, AVI, ARVI, ARVI2, BWDRVI, BRI, CARI, CARI2, Chlgreen, CIgreen, Chlrededge, CVI, CI, CTVI, CRI550, CRI700,\
          Datt1, Datt4, Datt6, D678500, D800550, D800680, DVIMSS, EVI2_, EPI, Fe2, Fe3, Fo, GEMI, GVMI, Gossan, GARI, GLI, GOSAVI, GSAVI, GBNDVI,\
          H, IVI, IPVI, I, IR550, IR700, LWCI, LogR, Maccioni, mCRIG, mCRIRE, MVI, MGVI, MNSI, MSBI, MYVI, mND680, mARI, MCARI1, MCARI2,\
          MCARIMTVI2, MCARIOSAVI, mSR, MSRNirRed, MSAVI, MSAVIhyper, MTVI1, MTVI2, NormG, NormR, NormNIR, PPR, PVR, GNDVIhyper, OSAVI, PNDVI, PVI_, RARSa1, RARSa2, RARSa3,\
          RARSa4, RARSc3, RARSc4, RBNDVI, REIP1, REIP2, Rre, RSR, SAVImir, IF, SR440740, SR520670, SR700, SR735710, SR774677, PSSRc1, SR8002170, SR860550, SR860708, SRMIRRed,\
          SRSWIRINIR, SAVI2, SARVI, SLAVI, SQRT_, SIPI_, GVIMSS, NSIMSS, SBIMSS, GVI, WET, YVIMSS, TSAVI, TSAVI2, TVI_, TCI, VARI700, WDVI, MNDVI, EPIChlb, LAI_green]

spectral_indices = ['MI', 'ARVI_', 'SIPI', 'RENDVI', 'MRESR', 'RYI', 'NDYI', 'DYI', 'ACI', 'AVI_', 'SI', 'BSI','SAVI', 'FIDET', 'MTCI_', 'NPCRI', 'S2REP', 'CCCI', 'MCARI', 'TCARI',\
                    'PVI', 'EVI', 'NDVI', 'BAI', 'NDSI', 'MRENDVI', 'NDVIre', 'CIre', 'NDMI', 'TNDVI', 'VDVI', 'NDVIc', 'TVI', 'EXG', 'PSRI', 'RDVI', 'RATIO1', 'RATIO2', 'RATIO3', 'RATIO4',\
                    'NDVI2', 'NDWI', 'NDRE', 'NDRE_', 'NDRE7', 'MSI', 'RECI', 'b7b5', 'b7b6', 'b8ab5', 'VARI', 'MTCI', 'ExBlue', 'WDRVI', 'GRNDVI', 'LCI', 'BNDVI', 'AFRI', 'aci', 'bai',\
                    'NLI', 'EVI2', 'ATSAVI', 'AFRI2100', 'ARI', 'AVI', 'ARVI', 'ARVI2', 'BWDRVI', 'BRI', 'CARI', 'CARI2', 'Chlgreen', 'CIgreen', 'Chlrededge', 'CVI', 'CI', 'CTVI', 'CRI550', 'CRI700',\
                    'Datt1', 'Datt4', 'Datt6', 'D678500', 'D800550', 'D800680', 'DVIMSS', 'EVI2_', 'EPI', 'Fe2', 'Fe3', 'Fo', 'GEMI', 'GVMI', 'Gossan', 'GARI', 'GLI', 'GOSAVI', 'GSAVI', 'GBNDVI',\
                    'H', 'IVI', 'IPVI', 'I', 'IR550', 'IR700', 'LWCI', 'LogR', 'Maccioni', 'mCRIG', 'mCRIRE', 'MVI', 'MGVI', 'MNSI', 'MSBI', 'MYVI', 'mND680', 'mARI', 'MCARI1', 'MCARI2',\
                    'MCARIMTVI2', 'MCARIOSAVI', 'mSR', 'MSRNirRed', 'MSAVI', 'MSAVIhyper', 'MTVI1', 'MTVI2', 'NormG', 'NormR', 'NormNIR', 'PPR', 'PVR', 'GNDVIhyper', 'OSAVI', 'PNDVI', 'PVI_', 'RARSa1', 'RARSa2', 'RARSa3',\
                    'RARSa4', 'RARSc3', 'RARSc4', 'RBNDVI', 'REIP1', 'REIP2', 'Rre', 'RSR', 'SAVImir', 'IF', 'SR440740', 'SR520670', 'SR700', 'SR735710', 'SR774677', 'PSSRc1', 'SR8002170', 'SR860550', 'SR860708', 'SRMIRRed',\
                    'SRSWIRINIR', 'SAVI2', 'SARVI', 'SLAVI', 'SQRT_', 'SIPI_', 'GVIMSS', 'NSIMSS', 'SBIMSS', 'GVI', 'WET', 'YVIMSS', 'TSAVI', 'TSAVI2', 'TVI_', 'TCI', 'VARI700', 'WDVI', 'MNDVI', 'EPIChlb', 'LAI_green']


In [ ]:
len(spectral_indices)

181

In [ ]:
# Create Features (or remote sensing indices) for 1 month

def vegetation_index_month(i):
  b01 = comb["B1"]
  b02 = comb["B2"]
  b03 = comb["B3"]
  b04 = comb["B4"]
  b05 = comb["B5"]
  b06 = comb["B6"]
  b07 = comb["B7"]
  b08 = comb["B8"]
  b8a = comb["B8A"]
  b09 = comb["B9"]
  b11 = comb["B11"]
  b12 = comb["B12"]

  MI = (b8a - b11) / (b8a + b11+ eps)
  ARVI_ =  (b08 - (2 * b04) + b02) / (b08 + (2 * b04) - b02+ eps)
  SIPI = (b08 - b02) / (b08 - b04+ eps)
  RENDVI = (b06 - b05) / (b06 + b05+ eps)
  MRESR = (b06 - b01) / (b05 - b01+ eps)
  RYI = b03 / (b02+ eps)
  NDYI = (b03 - b02) / (b03 + b02+ eps)
  DYI = b03 - b02
  ACI = b08 * (b04 + b03)
  AVI_ = (b08 * (1 - b04) * (b08 - b04))
  SI = ((1 - b02) * (1 - b03) * (1 - b04))
  BSI = ((b11 + b04) - (b08 + b02)) / ((b11 + b04) + (b08 + b02)+ eps)
  L = 0.33; SAVI = ((b08 - b04) / (b08 + b04 + L)) * (1 + L)
  FIDET = b12 / (b8a * b09+ eps)
  MTCI_ = (b06 - b05) / (b05 - b04+ eps)
  NPCRI = (b04 - b02) / (b04 + b02+ eps)
  S2REP = 705 + 35 * ((((b07 + b04) / 2) - b05) / (b06 - b05+ eps))
  CCCI = ((b08 - b05) / (b08 + b05+ eps)) / ((b08 - b04+ eps) / (b08 + b04+ eps))
  MCARI = ((b05 - b04) - 0.2 * (b05 - b03)) * (b05 / (b04+ eps))
  TCARI = 3 * ((b05 - b04) - 0.2 * (b05 - b03) * (b05 / (b04+ eps)))
  PVI = (b08 - 0.3 * b04 - 0.5) / ((1 + 0.3 * 2) ** (1 / 2.0))
  EVI = 2.5 * (b08 - b04) / (b08 + 6 * b04 - 7.5 * b02 + 1)
  EVI2 = 2.4 * (b08 - b04) / (b08 + b04 + 1)
  NDVI = ((b08 - b04) / (b08 + b04+ eps))
  NDVI2 = ((b12 - b08) / (b12 + b08+ eps))
  MNDVI = (b08 - b04) / (b08 + b04 - 2 * b02+ eps)
  BAI = 1 / ((0.1 - b04) ** 2 + (0.06 - b08) ** 2)
  NDSI = (b03 - b11) / (b03 + b11+ eps)
  MRENDVI = (b06 - b05) / (b06 + b05 - 2 * b01+ eps)
  NDVIre = (b08 - b05) / (b08 + b05+ eps)
  CIre = ((b08 / (b05+ eps)) - 1)
  NDMI = (b08 - b11) / (b08 + b11+ eps)
  TNDVI = [(x) ** (1 / 2.0) for x in ((b08 - b04) / (b08 + b04) + 0.5)]
  SWIRmin = 0.378; SWIRmax = 0.397; SWIIRmin = 0.027; NDVIc = (b08 - b04) / (b08 + b04) * (1.0 - (b12 - SWIIRmin) / (SWIRmax - SWIRmin))
  VDVI = (2 * b03 - b04 - b02) / (2 * b03 + b04 + b02+ eps)
  TVI = (120 * (b06 - b03) - 200 * (b04 - b03)) / 2
  EXG = 2 * b03 - b04 - b02
  PSRI = (b04 - b02) / (b06+ eps)
  RDVI = [(i - j) / (i + j) ** (1 / 2.0) for i, j in zip(b08, b04)]
  RATIO1 = b01 / (b03+ eps)
  RATIO2 = b01 / (b05+ eps)
  RATIO3 = b11 / (b12+ eps)
  RATIO4 = b05 / (b04+ eps)
  NDWI = (b03 - b08) / (b03 + b08+ eps)
  NDRE = (b09 - b05) / (b09 + b05+ eps)
  NDRE_ = (b05 - b04) / (b05 + b04+ eps)
  NDRE7 = (b07 - b04) / (b07 + b04+ eps)
  MSI = b11 / (b8a+ eps)
  RECI = (b08 / (b04+ eps)) - 1
  VARI = (b03 - b04) / (b03 + b04 - b02+ eps)
  MTCI = (b8a - b06) / (b07 + b06+ eps)
  ExBlue = (2 * b02) - (b03 + b04)
  WDRVI = (0.1 * b08 - b04) / (0.1 * b08 + b04+ eps)
  GRNDVI = (b08 - (b03 + b04)) / (b08 + (b03 + b04)+ eps)
  LCI = (b08 - b05) / (b08 + b04+ eps)
  BNDVI = (b08 - b02) / (b08 + b02+ eps)
  AFRI = b08 - 0.66 * (b11 / (b08 + 0.66 * b11+ eps))
  aci = b08 * (b04 + b03)
  bai = 1 / ((0.1 - b04) ** 2 + (0.06 - b08) ** 2)
  NLI = (b08 ** 2 - b04) / (b08 ** 2 + b04+ eps)
  b7b5 = (b07 - b05) / (b07 + b05+ eps)
  b7b6 = (b07 - b06) / (b07 + b06+ eps)
  b8ab5 = (b8a - b05) / (b8a + b05+ eps)
  ATSAVI = 1.22 * (b08 - 1.22 * b04 - 0.03) / (1.22 * b08 + b04 - 1.22 * 0.03 + 0.08 * (1.0 + np.power(1.22, 2.0)))
  AFRI2100 = b08 - 0.5 * b12 / (b08 + 0.56 * b12+ eps)
  ARI = 1.0 / (b03+ eps) - 1.0 / (b05+ eps)
  AVI = 2.0 * b09 - b04
  yy = 0.106; ARVI = (b8a - b04 - yy * (b04 - b02)) / (b8a + b04 - yy * (b04 - b02)+ eps)
  ARVI2 = -0.18 + 1.17 * ((b08 - b04) / (b08 + b04+ eps))
  BWDRVI = (0.1 * b08 - b02) / (0.1 * b08 + b02+ eps)
  BRI = (1.0 / (b03+ eps) - 1.0 / (b05+ eps)) / (b08+ eps)
  CARI = (b05 / (b04+ eps)) * (np.sqrt(np.power(((b05 - b03) / 150.0 * 670.0 + b04 + (b03 - ((b05 - b03) / 150.0 * 550.0))), 2.0))) / (np.power(((b05 - b03) / np.power(150.0, 2.0) + 1.0), 0.5))
  aaa = 0.496; CARI2 = (np.abs(((b05 - b03) / 150.0 * b04 + b04 + b03 - (aaa * b03))) / np.power((np.power(aaa, 2.0) + 1.0), 0.5)) * (b05 / b04)
  Chlgreen = np.power((b07 / (b03+ eps)), (-1.0))
  CIgreen = b08 / (b03+ eps) - 1.0
  Chlrededge = np.power((b07 / b05), (-1.0))
  CVI = b08 * b04 / np.power(b03, 2.0)
  CI = (b04 - b02) / (b04+ eps)
  CTVI = (((b04 - b03) / (b04 + b03)) + 0.5) / np.abs(((b04 - b03) / (b04 + b03)) + 0.5) * np.sqrt(np.abs((((b04 - b03) / (b04 + b03))) + 0.5))
  CRI550 = np.power(b02, (-1.0)) - np.power(b03, (-1.0))
  CRI700 = np.power(b02, (-1.0)) - np.power(b05, (-1.0))
  Datt1 = (b08 - b05) / (b08 - b04+ eps)
  Datt4 = b04 / ((b03 * b05)+ eps)
  Datt6 = b8a / ((b03 * b05)+ eps)
  D678500 = b04 - b02
  D800550 = b08 - b03
  D800680 = b08 - b04
  DVIMSS = 2.4 * b09 - b04
  EVI2_ = 2.5 * (b08 - b04) / (b08 + 2.4 * b04 + 1.0)
  aa = 0.331; bb = 0.329; EPI = aa * b04 / np.power((b03 * b05), bb)
  Fe2 = b12 / (b08+ eps) + b03 / (b04+ eps)
  Fe3 = b04 / (b03+ eps)
  Fo = b11 / (b08+ eps)
  GEMI = ((2.0 * (np.power(b08, 2.0) - np.power(b04, 2.0)) + 1.5 * b08 + 0.5 * b04) / (b08 + b04 + 0.5) * (1.0 - 0.25 * (2.0 * (np.power(b08, 2.0) - np.power(b04, 2.0)) + 1.5 * b08 + 0.5 * b04) / (b08 + b04 + 0.5)) - (b04 - 0.125) / (1.0 - b04))
  GVMI = ((b08 + 0.1) - (b12 + 0.02)) / ((b08 + 0.1) + (b12 + 0.02+ eps))
  Gossan = b11 / (b04+ eps)
  GARI = (b08 - (b03 - (b02 - b04))) / (b08 - (b03 + (b02 - b04))+ eps)
  GLI = (2.0 * b03 - b04 - b02) / (2.0 * b03 + b04 + b02+ eps)
  YY = 0.120; GOSAVI = (b08 - b03) / (b08 + b03 + YY)
  L = 0.482; GSAVI = (b08 - b03) / (b08 + b03 + L) * (1.0 + L)
  GBNDVI = (b08 - (b03 + b02)) / (b08 + (b03 + b02)+ eps)
  H = np.arctan((2.0 * b04 - b03 - b02) / 30.5 * (b03 - b02+ eps))
  a = 0.393; b = 0.809; IVI = (b08 - b) / (a * b04+ eps)
  IPVI = (b09 / (b09 + b05+ eps)) / 2 * (((b05 - b03) / (b05 + b03+ eps)) + 1)
  I = (1.0 / (30.5)) * (b04 + b03 + b02)
  IR550 = np.power(b03, (-1.0))
  IR700 = np.power(b05, (-1.0))
  MIDIR = 0.101; LWCI = np.log(1.0 - (b08 - MIDIR)) / (-np.log(1.0 - (b08 - MIDIR)))
  LogR = np.log(b08 / (b04+ eps))
  Maccioni = (b07 - b05) / (b07 - b04+ eps)
  mCRIG = (np.power(b02, (-1.0)) - np.power(b03, (-1.0))) * b08
  mCRIRE = (np.power(b02, (-1.0)) - np.power(b05, (-1.0))) * b08
  MVI = b09 / (b11+ eps)
  MGVI = -0.386 * b03 - 0.53 * b04 + 0.535 * b06 + 0.532 * b09
  MNSI = 0.404 * b03 - 0.039 * b04 - 0.505 * b06 + 0.762 * b09
  MSBI = 0.406 * b03 + 0.6 * b04 + 0.645 * b06 + 0.243 * b09
  MYVI = 0.723 * b03 - 0.597 * b04 + 0.206 * b06 - 0.278 * b09
  mND680 = (b08 - b04) / (b08 + b04 - 2.0 * b01+ eps)
  mARI = (np.power(b03, (-1.0)) - np.power(b05, (-1.0))) * b08
  MCARI1 = 1.2 * (2.5 * (b08 - b04) - 1.3 * (b08 - b03))
  MCARI2 = 1.5 * (2.5 * (b08 - b04) - 1.3 * (b08 - b03)) / np.sqrt(np.power((2.0 * b08 + 1.0), 2.0) - (6.0 * b08 - 5.0 * np.sqrt(b04)) - 0.5)
  MCARIMTVI2 = (((b05 - b04) - 0.2 * (b05 - b03)) * (b05 / (b04+ eps))) / (1.5 * (1.2 * (b08 - b03) - 2.5 * (b04 - b03)) / np.sqrt(np.power((2.0 * b08 + 1.0), 2.0) - (6.0 * b08 - 5.0 * np.sqrt(b04)) - 0.5))
  MCARIOSAVI = ((b05 - b04) - 0.2 * (b05 - b03) * (b05 / (b04+ eps))) / ((1.0 + 0.16) * (b08 - b04) / (b08 + b04 + 0.16))
  mSR = (b08 - b01) / (b04 - b01+ eps)
  MSRNirRed = ((b08 / (b04+ eps)) - 1.0) / np.sqrt((b08 / (b04+ eps)) + 1.0)
  MSAVI = (2.0 * b08 + 1.0 - np.sqrt(np.power((2.0 * b08 + 1.0), 2.0) - 8.0 * (b08 - b04))) / 2.0
  MSAVIhyper = (0.5) * ((2.0 * b08 + 1.0) - np.sqrt(np.power((2.0 * b08 + 1.0), 2.0) - 8.0 * (b08 - b04)))
  MTVI1 = 1.2 * (1.2 * (b08 - b03) - 2.5 * (b04 - b03))
  MTVI2 = 1.5 * (1.2 * (b08 - b03) - 2.5 * (b04 - b03)) / np.sqrt(np.power((2.0 * b08 + 1.0), 2.0) - (6.0 * b08 - 5.0 * np.sqrt(b04)) - 0.5)
  NormG = b03 / (b08 + b04 + b03+ eps)
  NormR = b04 / (b08 + b04 + b03+ eps)
  NormNIR = b08 / (b08 + b04 + b03+ eps)
  PPR = (b03 - b01) / (b03 + b01+ eps)
  PVR = (b03 - b04) / (b03 + b04+ eps)
  GNDVIhyper = (b07 - b03) / (b07 + b03+ eps)
  OSAVI = (1.0 + 0.16) * (b08 - b04) / (b08 + b04 + 0.16)
  PNDVI = (b08 - (b03 + b04 + b02)) / (b08 + (b03 + b04 + b02)+ eps)
  a = 0.149; ar = 0.374; b = 0.735; PVI_ = (1.0 / np.sqrt(a ** 2.0 + 1.0)) * (b08 - ar - b)
  r700 = 0.715; r675 = 0.722; RARSa1 = (b04 / b05) / (r675 / r700)
  r700 = 0.989; r680 = 0.848; RARSa2 = (b04 / b05) / (r680 / r700)
  r670 = 0.382; r800 = 0.935; RARSa3 = (b04 / b08) / (r670 / r800)
  r680 = 0.523; r800 = 0.866; RARSa4 = (b04 / b08) / (r680 / r800)
  r500 = 0.535; r800 = 0.712; RARSc3 = (b08 / b02) / (r800 / r500)
  r470 = 0.503; r800 = 0.153; RARSc4 = (b08 / b02) / (r800 / r470)
  RBNDVI = (b08 - (b04 + b02)) / (b08 + (b04 + b02)+ eps)
  REIP1 = 700.0 + 40.0 * ((((b04 + b07) / 2.0) - b05) / (b06 - b05+ eps))
  REIP2 = 702.0 + 40.0 * ((((b04 + b07) / 2.0) - b05) / (b06 - b05+ eps))
  Rre = (b04 + b07) / 2.0
  MIRmin = 0.259; MIRmax = 0.640; RSR = b08 / b04 * MIRmax - b12 / MIRmax - MIRmin
  L = 0.781; SAVImir = (b08 - b12) * (1.0 + L) / (b08 + b12 + L)
  IF = (2.0 * b04 - b03 - b02) / (b03 - b02+ eps)
  SR440740 = b01 / (b06+ eps)
  SR520670 = b02 / (b04+ eps)
  SR700 = 1.0 / (b05+ eps)
  SR735710 = b06 / (b05+ eps)
  SR774677 = b07 / (b04+ eps)
  PSSRc1 = b08 / (b02+ eps)
  SR8002170 = b08 / (b12+ eps)
  SR860550 = b8a / (b03+ eps)
  SR860708 = b8a / (b05+ eps)
  SRMIRRed = b12 / (b04+ eps)
  SWIRI = 0.887; SRSWIRINIR = SWIRI / (b08+ eps)
  a = 0.064; b = 0.918; SAVI2 = b08 / (b04 + b / a)
  y = 0.735; Rr = 0.740; L = 0.487; RB = 0.560; SARVI = (1.0 + L) * (b08 - (Rr - y * (RB - Rr))) / (b08 + -(Rr - y * (RB - Rr)) + L)
  SLAVI = b08 / (b04 + b12+ eps)
  SQRT_ = np.sqrt(b08 / b04+ eps)
  SIPI_ = (b08 - b01) / (b08 - b04+ eps)
  GVIMSS = -0.283 * b03 - 0.66 * b04 + 0.577 * b06 + 0.388 * b09
  NSIMSS = -0.016 * b03 + 0.131 * b04 - 0.425 * b06 + 0.882 * b09
  SBIMSS = 0.332 * b03 + 0.603 * b04 + 0.675 * b06 + 0.262 * b09
  GVI = -0.2848 * b02 - 0.2435 * b03 - 0.5436 * b04 + 0.7243 * b08 + 0.084 * b11 - 0.18 * b12
  WET = 0.1509 * b02 + 0.1973 * b03 + 0.3279 * b04 + 0.3406 * b08 - 0.7112 * b11 - 0.4572 * b12
  YVIMSS = -0.899 * b03 + 0.428 * b04 + 0.076 * b06 - 0.041 * b09
  X = 0.114; A = 0.824; B = 0.421; TSAVI = (B * (b08 - B * b04 - A)) / (b04 + B * (b08 - A) + X * (1.0 + np.power(B, 2.0))+ eps)
  a = 0.419; b = 0.787; TSAVI2 = (a * b08 - a * b04 - b) / (b04 + a * b08 - a * b+ eps)
  TVI_ = np.sqrt((((b04 - b03) / (b04 + b03+ eps))) + 0.5)
  TCI = 1.2 * (b05 - b03) - 1.5 * (b04 - b03) * np.sqrt(b05 / (b04+ eps))
  VARI700 = (b05 - 1.7 * b04 + 0.7 * b02) / (b05 + 2.3 * b04 - 1.3 * b02+ eps)
  a = 0.752; WDVI = b08 - a * b04
  EPIChlb = 0.0337 * np.power((b04 / (b03+ eps)), 1.8695)
  LAI_green = 5.405 * b8ab5 - 0.114

  return [MI, ARVI_, SIPI, RENDVI, MRESR, RYI, NDYI, DYI, ACI, AVI_, SI, BSI, SAVI, FIDET, MTCI_, NPCRI, S2REP, CCCI, MCARI, TCARI,\
          PVI, EVI, NDVI, BAI, NDSI, MRENDVI, NDVIre, CIre, NDMI, TNDVI, VDVI, NDVIc, TVI, EXG, PSRI, RDVI, RATIO1, RATIO2, RATIO3, RATIO4,\
          NDVI2, NDWI, NDRE, NDRE_, NDRE7, MSI, RECI, b7b5, b7b6, b8ab5, VARI, MTCI, ExBlue, WDRVI, GRNDVI, LCI, BNDVI, AFRI, aci, bai,\
          NLI, EVI2, ATSAVI, AFRI2100, ARI, AVI, ARVI, ARVI2, BWDRVI, BRI, CARI, CARI2, Chlgreen, CIgreen, Chlrededge, CVI, CI, CTVI, CRI550, CRI700,\
          Datt1, Datt4, Datt6, D678500, D800550, D800680, DVIMSS, EVI2_, EPI, Fe2, Fe3, Fo, GEMI, GVMI, Gossan, GARI, GLI, GOSAVI, GSAVI, GBNDVI,\
          H, IVI, IPVI, I, IR550, IR700, LWCI, LogR, Maccioni, mCRIG, mCRIRE, MVI, MGVI, MNSI, MSBI, MYVI, mND680, mARI, MCARI1, MCARI2,\
          MCARIMTVI2, MCARIOSAVI, mSR, MSRNirRed, MSAVI, MSAVIhyper, MTVI1, MTVI2, NormG, NormR, NormNIR, PPR, PVR, GNDVIhyper, OSAVI, PNDVI, PVI_, RARSa1, RARSa2, RARSa3,\
          RARSa4, RARSc3, RARSc4, RBNDVI, REIP1, REIP2, Rre, RSR, SAVImir, IF, SR440740, SR520670, SR700, SR735710, SR774677, PSSRc1, SR8002170, SR860550, SR860708, SRMIRRed,\
          SRSWIRINIR, SAVI2, SARVI, SLAVI, SQRT_, SIPI_, GVIMSS, NSIMSS, SBIMSS, GVI, WET, YVIMSS, TSAVI, TSAVI2, TVI_, TCI, VARI700, WDVI, MNDVI, EPIChlb, LAI_green]


# <font color="orange"> Best model for Afghanistan: Classification using Time Series of S1, S2 & L8


In [ ]:
train_data_S1S2 = train_data_afghanistan_S1S2
test_data_S1S2 = test_data_afghanistan_S1S2
train_data_L8 = train_data_afghanistan_landsat
test_data_L8 = test_data_afghanistan_landsat

In [ ]:
comb = train_data_S1S2.copy()

num_date = 12    # 12 month

vegs = Parallel(n_jobs=-1, verbose=2, backend="multiprocessing")(map(delayed(vegetation_index), [x for x in range(num_date)]))

data = {}
for i in range(num_date):
  for n,m in zip(spectral_indices, vegs[i]):
    data[f'PERIOD_{i}_{n}'] = m

vi_df = pd.DataFrame(data)

vi_df = vi_df.fillna(-999999).replace([np.inf, -np.inf], -999999)

indices_df = pd.merge(vi_df, comb, how = 'left', left_index=True, right_index=True)
indices_df.shape        # spectral indices: 181 * 12(months); main bands of S1 and S2: 15 * 12(months); ID,Lat,Lon,Target,index: 5

[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    1.1s finished


(500, 2357)

In [ ]:
comb = test_data_S1S2.copy()

num_date = 12

vegs = Parallel(n_jobs=-1, verbose=2, backend="multiprocessing")(map(delayed(vegetation_index), [x for x in range(num_date)]))

data_ = {}
for i in range(num_date):
  for n,m in zip(spectral_indices, vegs[i]):
    data_[f'PERIOD_{i}_{n}'] = m

vi_df_test = pd.DataFrame(data_)
vi_df_test = vi_df_test.fillna(-999999).replace([np.inf, -np.inf], -999999)

vi_df_test2 = pd.merge(vi_df_test, comb, how = 'left', left_index=True, right_index=True)
vi_df_test2.shape   # spectral indices: 181 * 12(months); main bands of S1 and S2: 15 * 12(months); ID,Lat,Lon,index: 4

[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    0.4s finished


(500, 2356)

In [ ]:
features = ['B1','B2','B3','B4','B5','B6','B7','B10','B11']
months = [f'{feat}_month{i}' for i in range(1,13) for feat in features]

df = train_data_L8
df_train = pd.DataFrame()
df_month = pd.DataFrame()

for month_num in range(1,13):

  df_month = df[months[(month_num-1)*len(features):month_num*len(features)]].copy()
  df_month.columns = [f"{feat}_month{month_num}" for feat in features]

  df_month[f'NDVI_month{month_num}'] = (df_month[f'B5_month{month_num}'] - df_month[f'B4_month{month_num}']) / (df_month[f'B5_month{month_num}'] + df_month[f'B4_month{month_num}'])
  df_month[f'NDWI_month{month_num}'] = (df_month[f'B3_month{month_num}'] - df_month[f'B5_month{month_num}']) / (df_month[f'B3_month{month_num}'] + df_month[f'B5_month{month_num}'])
  df_month[f'EVI_month{month_num}'] = 2.5 * (df_month[f'B5_month{month_num}'] - df_month[f'B4_month{month_num}']) / (df_month[f'B5_month{month_num}'] + 6 * df_month[f'B4_month{month_num}'] - 7.5 * df_month[f'B2_month{month_num}'] + 1)
  df_month[f'BUI_month{month_num}'] = (df_month[f'B6_month{month_num}'] + df_month[f'B5_month{month_num}']) - (df_month[f'B4_month{month_num}'] + df_month[f'B3_month{month_num}'])
  df_month[f'NDBI_month{month_num}'] = (df_month[f'B6_month{month_num}'] - df_month[f'B5_month{month_num}']) / (df_month[f'B6_month{month_num}'] + df_month[f'B5_month{month_num}'])
  df_month[f'NDSI_month{month_num}'] = (df_month[f'B3_month{month_num}'] - df_month[f'B6_month{month_num}']) / (df_month[f'B3_month{month_num}'] + df_month[f'B6_month{month_num}'])
  df_month[f'SAVI_month{month_num}'] = (1 + 0.5) * (df_month[f'B5_month{month_num}'] - df_month[f'B4_month{month_num}']) / (df_month[f'B5_month{month_num}'] + df_month[f'B4_month{month_num}'] + 0.5)

  df_train = pd.concat([df_train, df_month], axis=1)

df = test_data_L8
df_test = pd.DataFrame()
df_month = pd.DataFrame()

for month_num in range(1,13):

  df_month = df[months[(month_num-1)*len(features):month_num*len(features)]].copy()
  df_month.columns = [f"{feat}_month{month_num}" for feat in features]

  df_month[f'NDVI_month{month_num}'] = (df_month[f'B5_month{month_num}'] - df_month[f'B4_month{month_num}']) / (df_month[f'B5_month{month_num}'] + df_month[f'B4_month{month_num}'])
  df_month[f'NDWI_month{month_num}'] = (df_month[f'B3_month{month_num}'] - df_month[f'B5_month{month_num}']) / (df_month[f'B3_month{month_num}'] + df_month[f'B5_month{month_num}'])
  df_month[f'EVI_month{month_num}'] = 2.5 * (df_month[f'B5_month{month_num}'] - df_month[f'B4_month{month_num}']) / (df_month[f'B5_month{month_num}'] + 6 * df_month[f'B4_month{month_num}'] - 7.5 * df_month[f'B2_month{month_num}'] + 1)
  df_month[f'BUI_month{month_num}'] = (df_month[f'B6_month{month_num}'] + df_month[f'B5_month{month_num}']) - (df_month[f'B4_month{month_num}'] + df_month[f'B3_month{month_num}'])
  df_month[f'NDBI_month{month_num}'] = (df_month[f'B6_month{month_num}'] - df_month[f'B5_month{month_num}']) / (df_month[f'B6_month{month_num}'] + df_month[f'B5_month{month_num}'])
  df_month[f'NDSI_month{month_num}'] = (df_month[f'B3_month{month_num}'] - df_month[f'B6_month{month_num}']) / (df_month[f'B3_month{month_num}'] + df_month[f'B6_month{month_num}'])
  df_month[f'SAVI_month{month_num}'] = (1 + 0.5) * (df_month[f'B5_month{month_num}'] - df_month[f'B4_month{month_num}']) / (df_month[f'B5_month{month_num}'] + df_month[f'B4_month{month_num}'] + 0.5)

  df_test = pd.concat([df_test, df_month], axis=1)

In [ ]:
indices_df = pd.merge(indices_df, df_train, how = 'left', left_index=True, right_index=True)
vi_df_test2 = pd.merge(vi_df_test2, df_test, how = 'left', left_index=True, right_index=True)

In [ ]:
s_train = indices_df.drop(['index', 'ID', 'Lat',	'Lon', 'Target'], axis=1)
target_season = indices_df['Target']
s_test = vi_df_test2.drop(['index', 'ID', 'Lat',	'Lon'], axis=1)

In [ ]:
X = s_train.copy()
y = target_season.copy()
tess_ = s_test.copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED)

In [ ]:
# Best model for afghanistan
CAT = CatBoostClassifier(iterations=200, learning_rate=0.02, depth=6, bagging_temperature = 0.3, random_seed=SEED)

In [ ]:
model = CAT
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'Accuracy Score: {accuracy_score(y_test, y_pred)}')
print('LOSS: ', log_loss(y_test, y_pred))

0:	learn: 0.6781142	total: 1.95s	remaining: 6m 28s
1:	learn: 0.6630806	total: 3.11s	remaining: 5m 8s
2:	learn: 0.6498568	total: 4.21s	remaining: 4m 36s
3:	learn: 0.6369495	total: 5.22s	remaining: 4m 15s
4:	learn: 0.6256818	total: 5.85s	remaining: 3m 48s
5:	learn: 0.6144142	total: 6.47s	remaining: 3m 29s
6:	learn: 0.6004344	total: 7.09s	remaining: 3m 15s
7:	learn: 0.5914356	total: 7.7s	remaining: 3m 4s
8:	learn: 0.5798303	total: 8.3s	remaining: 2m 56s
9:	learn: 0.5700620	total: 8.9s	remaining: 2m 49s
10:	learn: 0.5572733	total: 9.49s	remaining: 2m 43s
11:	learn: 0.5468446	total: 10.1s	remaining: 2m 38s
12:	learn: 0.5361706	total: 10.7s	remaining: 2m 34s
13:	learn: 0.5257546	total: 11.3s	remaining: 2m 30s
14:	learn: 0.5169410	total: 12s	remaining: 2m 27s
15:	learn: 0.5055321	total: 12.6s	remaining: 2m 24s
16:	learn: 0.4969926	total: 13.2s	remaining: 2m 21s
17:	learn: 0.4853408	total: 13.8s	remaining: 2m 19s
18:	learn: 0.4765554	total: 14.4s	remaining: 2m 17s
19:	learn: 0.4670180	total: 1

In [ ]:
predictions = model.predict(tess_)
sub_file = pd.DataFrame({'ID': test_data_S1S2.ID, 'Target': predictions})
sub_file.to_csv('afghanistan.csv', index = False)

# <font color="orange"> Best model for Sudan: Classification using Time Series of S1, S2 & L8

In [ ]:
train_data_S1S2 = train_data_sudan_S1S2
test_data_S1S2 = test_data_sudan_S1S2
train_data_L8 = train_data_sudan_landsat
test_data_L8 = test_data_sudan_landsat

In [ ]:
comb = train_data_S1S2.copy()

num_date = 12    # 12 month

vegs = Parallel(n_jobs=-1, verbose=2, backend="multiprocessing")(map(delayed(vegetation_index), [x for x in range(num_date)]))

data = {}
for i in range(num_date):
  for n,m in zip(spectral_indices, vegs[i]):
    data[f'PERIOD_{i}_{n}'] = m

vi_df = pd.DataFrame(data)

vi_df = vi_df.fillna(-999999).replace([np.inf, -np.inf], -999999)
indices_df = pd.merge(vi_df, comb, how = 'left', left_index=True, right_index=True)
indices_df.shape   # spectral indices: 181 * 12(months); main bands of S1 and S2: 14 * 12(months); ID,Lat,Lon,Target,index: 5

[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    0.4s finished


(500, 2345)

In [ ]:
comb = test_data_S1S2.copy()

num_date = 12

vegs = Parallel(n_jobs=-1, verbose=2, backend="multiprocessing")(map(delayed(vegetation_index), [x for x in range(num_date)]))

data_ = {}
for i in range(num_date):
  for n,m in zip(spectral_indices, vegs[i]):
    data_[f'PERIOD_{i}_{n}'] = m

vi_df_test = pd.DataFrame(data_)
vi_df_test = vi_df_test.fillna(-999999).replace([np.inf, -np.inf], -999999)
vi_df_test2 = pd.merge(vi_df_test, comb, how = 'left', left_index=True, right_index=True)
vi_df_test2.shape   # spectral indices: 181 * 12(months); main bands of S1 and S2: 14 * 12(months); ID,Lat,Lon,index: 4

[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    0.4s finished


(500, 2344)

In [ ]:
features = ['B1','B2','B3','B4','B5','B6','B7','B10','B11']
months = [f'{feat}_month{i}' for i in range(1,13) for feat in features]

df = train_data_L8
df_train = pd.DataFrame()
df_month = pd.DataFrame()

for month_num in range(1,13):

  df_month = df[months[(month_num-1)*len(features):month_num*len(features)]].copy()
  df_month.columns = [f"{feat}_month{month_num}" for feat in features]

  df_month[f'NDVI_month{month_num}'] = (df_month[f'B5_month{month_num}'] - df_month[f'B4_month{month_num}']) / (df_month[f'B5_month{month_num}'] + df_month[f'B4_month{month_num}'])
  df_month[f'NDWI_month{month_num}'] = (df_month[f'B3_month{month_num}'] - df_month[f'B5_month{month_num}']) / (df_month[f'B3_month{month_num}'] + df_month[f'B5_month{month_num}'])
  df_month[f'EVI_month{month_num}'] = 2.5 * (df_month[f'B5_month{month_num}'] - df_month[f'B4_month{month_num}']) / (df_month[f'B5_month{month_num}'] + 6 * df_month[f'B4_month{month_num}'] - 7.5 * df_month[f'B2_month{month_num}'] + 1)
  df_month[f'BUI_month{month_num}'] = (df_month[f'B6_month{month_num}'] + df_month[f'B5_month{month_num}']) - (df_month[f'B4_month{month_num}'] + df_month[f'B3_month{month_num}'])
  df_month[f'NDBI_month{month_num}'] = (df_month[f'B6_month{month_num}'] - df_month[f'B5_month{month_num}']) / (df_month[f'B6_month{month_num}'] + df_month[f'B5_month{month_num}'])
  df_month[f'NDSI_month{month_num}'] = (df_month[f'B3_month{month_num}'] - df_month[f'B6_month{month_num}']) / (df_month[f'B3_month{month_num}'] + df_month[f'B6_month{month_num}'])
  df_month[f'SAVI_month{month_num}'] = (1 + 0.5) * (df_month[f'B5_month{month_num}'] - df_month[f'B4_month{month_num}']) / (df_month[f'B5_month{month_num}'] + df_month[f'B4_month{month_num}'] + 0.5)

  df_train = pd.concat([df_train, df_month], axis=1)

df = test_data_L8
df_test = pd.DataFrame()
df_month = pd.DataFrame()

for month_num in range(1,13):

  df_month = df[months[(month_num-1)*len(features):month_num*len(features)]].copy()
  df_month.columns = [f"{feat}_month{month_num}" for feat in features]

  df_month[f'NDVI_month{month_num}'] = (df_month[f'B5_month{month_num}'] - df_month[f'B4_month{month_num}']) / (df_month[f'B5_month{month_num}'] + df_month[f'B4_month{month_num}'])
  df_month[f'NDWI_month{month_num}'] = (df_month[f'B3_month{month_num}'] - df_month[f'B5_month{month_num}']) / (df_month[f'B3_month{month_num}'] + df_month[f'B5_month{month_num}'])
  df_month[f'EVI_month{month_num}'] = 2.5 * (df_month[f'B5_month{month_num}'] - df_month[f'B4_month{month_num}']) / (df_month[f'B5_month{month_num}'] + 6 * df_month[f'B4_month{month_num}'] - 7.5 * df_month[f'B2_month{month_num}'] + 1)
  df_month[f'BUI_month{month_num}'] = (df_month[f'B6_month{month_num}'] + df_month[f'B5_month{month_num}']) - (df_month[f'B4_month{month_num}'] + df_month[f'B3_month{month_num}'])
  df_month[f'NDBI_month{month_num}'] = (df_month[f'B6_month{month_num}'] - df_month[f'B5_month{month_num}']) / (df_month[f'B6_month{month_num}'] + df_month[f'B5_month{month_num}'])
  df_month[f'NDSI_month{month_num}'] = (df_month[f'B3_month{month_num}'] - df_month[f'B6_month{month_num}']) / (df_month[f'B3_month{month_num}'] + df_month[f'B6_month{month_num}'])
  df_month[f'SAVI_month{month_num}'] = (1 + 0.5) * (df_month[f'B5_month{month_num}'] - df_month[f'B4_month{month_num}']) / (df_month[f'B5_month{month_num}'] + df_month[f'B4_month{month_num}'] + 0.5)

  df_test = pd.concat([df_test, df_month], axis=1)

In [ ]:
indices_df = pd.merge(indices_df, df_train, how = 'left', left_index=True, right_index=True)
vi_df_test2 = pd.merge(vi_df_test2, df_test, how = 'left', left_index=True, right_index=True)

s_train = indices_df.drop(['index', 'ID', 'Lat',	'Lon', 'Target'], axis=1)
target_season = indices_df['Target']
s_test = vi_df_test2.drop(['index', 'ID', 'Lat',	'Lon'], axis=1)

In [ ]:
X = s_train.copy()
y = target_season.copy()
tess_ = s_test.copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED)

In [ ]:
# Best model for Sudan
CAT = CatBoostClassifier(iterations=200, learning_rate=0.02, depth=6, bagging_temperature = 0.3, random_seed=SEED)

In [ ]:
model = CAT
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'Accuracy Score: {accuracy_score(y_test, y_pred)}')
print('LOSS: ', log_loss(y_test, y_pred))

0:	learn: 0.6705449	total: 837ms	remaining: 2m 46s
1:	learn: 0.6434977	total: 1.44s	remaining: 2m 22s
2:	learn: 0.6190199	total: 2.07s	remaining: 2m 15s
3:	learn: 0.5915477	total: 2.67s	remaining: 2m 10s
4:	learn: 0.5707429	total: 3.28s	remaining: 2m 7s
5:	learn: 0.5492892	total: 3.89s	remaining: 2m 5s
6:	learn: 0.5286369	total: 4.5s	remaining: 2m 4s
7:	learn: 0.5097023	total: 5.11s	remaining: 2m 2s
8:	learn: 0.4914133	total: 5.72s	remaining: 2m 1s
9:	learn: 0.4700387	total: 6.32s	remaining: 2m
10:	learn: 0.4542614	total: 6.91s	remaining: 1m 58s
11:	learn: 0.4367108	total: 7.5s	remaining: 1m 57s
12:	learn: 0.4213186	total: 8.57s	remaining: 2m 3s
13:	learn: 0.4041178	total: 9.71s	remaining: 2m 8s
14:	learn: 0.3832322	total: 10.8s	remaining: 2m 13s
15:	learn: 0.3693160	total: 11.9s	remaining: 2m 17s
16:	learn: 0.3554939	total: 12.5s	remaining: 2m 14s
17:	learn: 0.3416643	total: 13.1s	remaining: 2m 12s
18:	learn: 0.3306947	total: 13.7s	remaining: 2m 10s
19:	learn: 0.3187391	total: 14.3s	r

In [ ]:
predictions = model.predict(tess_)
sub_file = pd.DataFrame({'ID': test_data_S1S2.ID, 'Target': predictions})
sub_file.to_csv('sudan.csv', index = False)

# <font color="orange"> Best model for Iran: Classification using Time Series of S1, S2 & L8

In [ ]:
train_data_S1S2 = train_data_iran_S1S2
test_data_S1S2 = test_data_iran_S1S2
train_data_L8 = train_data_iran_landsat
test_data_L8 = test_data_iran_landsat

In [ ]:
comb = train_data_S1S2.copy()

num_date = 12    # 12 month

vegs = Parallel(n_jobs=-1, verbose=2, backend="multiprocessing")(map(delayed(vegetation_index), [x for x in range(num_date)]))

data = {}
for i in range(num_date):
  for n,m in zip(spectral_indices, vegs[i]):
    data[f'PERIOD_{i}_{n}'] = m

vi_df = pd.DataFrame(data)

vi_df = vi_df.fillna(-999999).replace([np.inf, -np.inf], -999999)

indices_df = pd.merge(vi_df, comb, how = 'left', left_index=True, right_index=True)
indices_df.shape        # spectral indices: 181 * 12(months); main bands of S1 and S2: 15 * 12(months); ID,Lat,Lon,Target,index: 5

[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    0.4s finished


(500, 2357)

In [ ]:
comb = test_data_S1S2.copy()

num_date = 12

vegs = Parallel(n_jobs=-1, verbose=2, backend="multiprocessing")(map(delayed(vegetation_index), [x for x in range(num_date)]))

data_ = {}
for i in range(num_date):
  for n,m in zip(spectral_indices, vegs[i]):
    data_[f'PERIOD_{i}_{n}'] = m

vi_df_test = pd.DataFrame(data_)
vi_df_test = vi_df_test.fillna(-999999).replace([np.inf, -np.inf], -999999)

vi_df_test2 = pd.merge(vi_df_test, comb, how = 'left', left_index=True, right_index=True)
vi_df_test2.shape   # spectral indices: 181 * 12(months); main bands of S1 and S2: 15 * 12(months); ID,Lat,Lon,index: 4

[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    0.4s finished


(500, 2356)

In [ ]:
features = ['B1','B2','B3','B4','B5','B6','B7','B10','B11']
months = [f'{feat}_month{i}' for i in range(1,13) for feat in features]

df = train_data_L8
df_train = pd.DataFrame()
df_month = pd.DataFrame()

for month_num in range(1,13):

  df_month = df[months[(month_num-1)*len(features):month_num*len(features)]].copy()
  df_month.columns = [f"{feat}_month{month_num}" for feat in features]

  df_month[f'NDVI_month{month_num}'] = (df_month[f'B5_month{month_num}'] - df_month[f'B4_month{month_num}']) / (df_month[f'B5_month{month_num}'] + df_month[f'B4_month{month_num}'])
  df_month[f'NDWI_month{month_num}'] = (df_month[f'B3_month{month_num}'] - df_month[f'B5_month{month_num}']) / (df_month[f'B3_month{month_num}'] + df_month[f'B5_month{month_num}'])
  df_month[f'EVI_month{month_num}'] = 2.5 * (df_month[f'B5_month{month_num}'] - df_month[f'B4_month{month_num}']) / (df_month[f'B5_month{month_num}'] + 6 * df_month[f'B4_month{month_num}'] - 7.5 * df_month[f'B2_month{month_num}'] + 1)
  df_month[f'BUI_month{month_num}'] = (df_month[f'B6_month{month_num}'] + df_month[f'B5_month{month_num}']) - (df_month[f'B4_month{month_num}'] + df_month[f'B3_month{month_num}'])
  df_month[f'NDBI_month{month_num}'] = (df_month[f'B6_month{month_num}'] - df_month[f'B5_month{month_num}']) / (df_month[f'B6_month{month_num}'] + df_month[f'B5_month{month_num}'])
  df_month[f'NDSI_month{month_num}'] = (df_month[f'B3_month{month_num}'] - df_month[f'B6_month{month_num}']) / (df_month[f'B3_month{month_num}'] + df_month[f'B6_month{month_num}'])
  df_month[f'SAVI_month{month_num}'] = (1 + 0.5) * (df_month[f'B5_month{month_num}'] - df_month[f'B4_month{month_num}']) / (df_month[f'B5_month{month_num}'] + df_month[f'B4_month{month_num}'] + 0.5)

  df_train = pd.concat([df_train, df_month], axis=1)

df = test_data_L8
df_test = pd.DataFrame()
df_month = pd.DataFrame()

for month_num in range(1,13):

  df_month = df[months[(month_num-1)*len(features):month_num*len(features)]].copy()
  df_month.columns = [f"{feat}_month{month_num}" for feat in features]

  df_month[f'NDVI_month{month_num}'] = (df_month[f'B5_month{month_num}'] - df_month[f'B4_month{month_num}']) / (df_month[f'B5_month{month_num}'] + df_month[f'B4_month{month_num}'])
  df_month[f'NDWI_month{month_num}'] = (df_month[f'B3_month{month_num}'] - df_month[f'B5_month{month_num}']) / (df_month[f'B3_month{month_num}'] + df_month[f'B5_month{month_num}'])
  df_month[f'EVI_month{month_num}'] = 2.5 * (df_month[f'B5_month{month_num}'] - df_month[f'B4_month{month_num}']) / (df_month[f'B5_month{month_num}'] + 6 * df_month[f'B4_month{month_num}'] - 7.5 * df_month[f'B2_month{month_num}'] + 1)
  df_month[f'BUI_month{month_num}'] = (df_month[f'B6_month{month_num}'] + df_month[f'B5_month{month_num}']) - (df_month[f'B4_month{month_num}'] + df_month[f'B3_month{month_num}'])
  df_month[f'NDBI_month{month_num}'] = (df_month[f'B6_month{month_num}'] - df_month[f'B5_month{month_num}']) / (df_month[f'B6_month{month_num}'] + df_month[f'B5_month{month_num}'])
  df_month[f'NDSI_month{month_num}'] = (df_month[f'B3_month{month_num}'] - df_month[f'B6_month{month_num}']) / (df_month[f'B3_month{month_num}'] + df_month[f'B6_month{month_num}'])
  df_month[f'SAVI_month{month_num}'] = (1 + 0.5) * (df_month[f'B5_month{month_num}'] - df_month[f'B4_month{month_num}']) / (df_month[f'B5_month{month_num}'] + df_month[f'B4_month{month_num}'] + 0.5)

  df_test = pd.concat([df_test, df_month], axis=1)

In [ ]:
indices_df = pd.merge(indices_df, df_train, how = 'left', left_index=True, right_index=True)
vi_df_test2 = pd.merge(vi_df_test2, df_test, how = 'left', left_index=True, right_index=True)

In [ ]:
s_train = indices_df.drop(['index', 'ID', 'Lat',	'Lon', 'Target'], axis=1)
target_season = indices_df['Target']
s_test = vi_df_test2.drop(['index', 'ID', 'Lat',	'Lon'], axis=1)

In [ ]:
X = s_train.copy()
y = target_season.copy()
tess_ = s_test.copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED)

In [ ]:
# Best model for Iran
CAT = CatBoostClassifier(iterations=500, learning_rate=0.04, depth=2, bagging_temperature = 0.3, random_seed=SEED)

In [ ]:
model = CAT
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'Accuracy Score: {accuracy_score(y_test, y_pred)}')
print('LOSS: ', log_loss(y_test, y_pred))

0:	learn: 0.6521524	total: 98.7ms	remaining: 49.3s
1:	learn: 0.6221644	total: 179ms	remaining: 44.5s
2:	learn: 0.5901871	total: 264ms	remaining: 43.8s
3:	learn: 0.5612935	total: 353ms	remaining: 43.7s
4:	learn: 0.5361812	total: 447ms	remaining: 44.2s
5:	learn: 0.5064647	total: 535ms	remaining: 44.1s
6:	learn: 0.4866987	total: 618ms	remaining: 43.5s
7:	learn: 0.4629169	total: 703ms	remaining: 43.3s
8:	learn: 0.4470280	total: 783ms	remaining: 42.7s
9:	learn: 0.4305922	total: 869ms	remaining: 42.6s
10:	learn: 0.4117903	total: 960ms	remaining: 42.7s
11:	learn: 0.3961612	total: 1.04s	remaining: 42.3s
12:	learn: 0.3744721	total: 1.12s	remaining: 42.1s
13:	learn: 0.3610420	total: 1.21s	remaining: 42s
14:	learn: 0.3430945	total: 1.29s	remaining: 41.8s
15:	learn: 0.3320323	total: 1.38s	remaining: 41.8s
16:	learn: 0.3180835	total: 1.47s	remaining: 41.9s
17:	learn: 0.3058002	total: 1.55s	remaining: 41.6s
18:	learn: 0.2928724	total: 1.65s	remaining: 41.7s
19:	learn: 0.2833031	total: 1.73s	remainin

In [ ]:
predictions = model.predict(tess_)
sub_file = pd.DataFrame({'ID': test_data_S1S2.ID, 'Target': predictions})
sub_file.to_csv('iran.csv', index = False)

# <font color="brown"> Final model

In [ ]:
# Load best models
model1 = pd.read_csv('afghanistan.csv')
model2 = pd.read_csv('sudan.csv')
model3 = pd.read_csv('iran.csv')

In [ ]:
Final_model = pd.concat([model3, model2, model1], axis=0)
Final_model.to_csv('Final_model.csv', index = False)